In [17]:
import os
import re
import glob
import csv

In [18]:
!wget https://github.com/vs8th/reports/archive/main.zip

import zipfile

with zipfile.ZipFile("main.zip", 'r') as zip_ref:
    zip_ref.extractall()

--2025-02-10 23:48:46--  https://github.com/vs8th/reports/archive/main.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Vs8th/reports/zip/refs/heads/main [following]
--2025-02-10 23:48:47--  https://codeload.github.com/Vs8th/reports/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: 'main.zip'

     0K ...                                                    22.0M=0s

2025-02-10 23:48:47 (22.0 MB/s) - 'main.zip' saved [3296]



In [19]:
def is_sublist(lst, sub):
    sub_len = len(sub)
    for i in range(len(lst) - sub_len + 1):
        if lst[i:i+sub_len] == sub:
            return i
    return -1

In [20]:
def process_files(source_dir, destination_dir):

    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # фильтр
    pattern = re.compile(r'^\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d+\.csv$')
    
    all_files = glob.glob(os.path.join(source_dir, '*'))
    csv_files = [file for file in all_files if pattern.match(os.path.basename(file))]
    
    if not csv_files:
        print("Не найдено файлов, по заданному шаблону.")
        return
    
    # Сортируем (не понятно: надо - не надо)
    csv_files.sort()
    
    # Путь к результирующему файлу
    combined_file_path = os.path.join(destination_dir, 'combined_data.csv')
    
    header_written = False
    header_init = []
    output_list = []

    for file in csv_files:
        with open(file, 'r', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile, delimiter=';')
            header = next(reader)
            sublist_index = 0

            if not header_written:
                output_list.append(header)
                header_written = True
                header_init = list(header)
            else:
                sublist_index = is_sublist(header, header_init)
                if sublist_index < 0:
                    continue
            
            for row in reader:
                output_list.append(row[sublist_index:])
                    
    with open(combined_file_path, 'w', encoding='utf-8', newline='') as outfile:
        writer = csv.writer(outfile, delimiter=',', quotechar='"')
        for row in output_list:
            writer.writerow(row)


src_folder = 'reports-main'
dest_folder = 'comb_reports'
process_files(src_folder, dest_folder)

In [21]:
!wget https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv

import pandas as pd

user_answer = pd.read_csv(f'{dest_folder}/combined_data.csv')
correct_answer = pd.read_csv('data.csv')

--2025-02-10 23:48:47--  https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv [following]
--2025-02-10 23:48:47--  https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984 [text/plain]
Saving to: 'data.csv'

     0K                                                       100% 98.8M=0s

2025-02-10 23:48:47 (98.8 MB/s) - 'data.csv' saved [984/984]



In [22]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')

Поздравляем, Вы справились и успешно прошли все проверки!
